In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
## load data 
hhdf = pd.read_excel('./in/shocks_dataset_fji_2704.xlsx')
print(hhdf.columns)

indiv = pd.read_excel('./in/shocks_dataset_fji_2704_indiv_v2.xlsx')
print('\n')
print(indiv.columns)


Index(['hhid', 'totweight', 'urban', 'inc_employ', 'inc_commerce',
       'inc_subsist', 'inc_remit', 'inc_transfer', 'inc_gift', 'inc_rent',
       'inc_total', 'food_cons', 'total_cons', 'poor_nat', 'poor_190',
       'poor_320', 'poor_550', 'dep_water', 'dep_san', 'dep_ele', 'dep_enroll',
       'dep_attain'],
      dtype='object')


Index(['hhid', 'indid', 'age', 'educ', 'employ', 'ind_isic', 'inc_emp_ind',
       'fji_conv'],
      dtype='object')


# Quick look at poverty headcount

In [3]:
## add household size to the columns
hhsize_dict = indiv.groupby('hhid')['indid'].count().to_dict()
hhdf['hhsize'] = hhdf['hhid'].map(hhsize_dict)


## calculating number of children in household (below 18)
indiv['ischild'] = [1 if x<=18 else 0 for x in list(indiv['age'])]
number_of_children_dict = indiv.groupby('hhid')['ischild'].sum().to_dict()
hhdf['number_of_children'] = hhdf['hhid'].map(number_of_children_dict)

#calculating percapita consumption (per adult equivalent) to get households falling below national poverty line of $2,179.39 per year per adult
# so pcexp = (total_consumption*hhwgt)/hhsize

hhdf['pcexp'] = (hhdf['total_cons']/(hhdf['hhsize']-hhdf['number_of_children']))
hhdf['pcinc'] = hhdf['inc_total']/(hhdf['hhsize']-hhdf['number_of_children'])
hhdf['ispoor'] = [1 if x<=2179.39 else 0 for x in list(hhdf['pcinc'])]

print('total population is: ',(hhdf['totweight'].sum()).round(0),'\n')

print('When considering only adults to calculate per capita income per adult equivalent')

print('poverty headcount: ',(hhdf['ispoor']*hhdf['totweight']).sum().round(0))
print('poverty rate: ',((hhdf['ispoor']*hhdf['totweight']).sum()/(hhdf['totweight'].sum()).round(1)*100),'% \n')


print('When considering all members of household to calculate percapita income')

hhdf['pcexp'] = (hhdf['total_cons']/(hhdf['hhsize']))
hhdf['pcinc'] = hhdf['inc_total']/(hhdf['hhsize'])
hhdf['ispoor'] = [1 if x<=2179.39 else 0 for x in list(hhdf['pcinc'])]
print('poverty headcount: ',(hhdf['ispoor']*hhdf['totweight']).sum().round(0))
print('poverty rate (all): ',((hhdf['ispoor']*hhdf['totweight']).sum()/(hhdf['totweight'].sum())).round(1)*100,'% \n')



total population is:  864132.0 

When considering only adults to calculate per capita income per adult equivalent
poverty headcount:  27378.0
poverty rate:  3.1683211093753196 % 

When considering all members of household to calculate percapita income
poverty headcount:  171438.0
poverty rate (all):  20.0 % 



## Preparing the households dataframe

In [4]:
## to keep adults within the working age so as to avoid getting older people simply because of the max age
#indiv = indiv[(indiv['age']>=18)&(indiv['age']<=65)]
## dropping the method above. 

## instead, getting the head of household from indid = 1
hoh = indiv.loc[indiv['indid']==1].set_index('hhid')

## adding columns to help with income profiling
hhdf['hoh_mainactivity']= hhdf['hhid'].map(hoh['employ'].to_dict())
hhdf['hoh_education']=hhdf['hhid'].map(hoh['educ'].to_dict())

##getting the mainsource of income for each household
hhdf['hhinc_mainsource_all']=hhdf[['inc_employ', 'inc_commerce','inc_subsist', 'inc_remit', 'inc_transfer', 'inc_gift', 'inc_rent']].idxmax(axis=1)
temp = {'inc_employ':'inc_employ',
        'inc_commerce':'other',
        'inc_subsist':'other',
         'inc_remit':'other', 
         'inc_transfer':'other', 
         'inc_gift':'other',
          'inc_rent':'other'}

hhdf['hhinc_mainsource_wage_other'] = hhdf['hhinc_mainsource_all'].map(temp)


## list of household ids where there is no head of household info. 
missing_hoh = list(hhdf[hhdf['hoh_mainactivity'].isna()]['hhid'])

## pulling up the data from the main individuals dataframe
indiv[indiv['hhid'].isin(missing_hoh)]
## checking if there are any indid=1 conditions satisfied
indiv[indiv['hhid'].isin(missing_hoh)][indiv['indid']==1]
## this reveals the problem: there are 21 rows without indid=1

## for all the residents of these households, the residents remain within the working age. 
## therefore it is safe to use age.max() and assume they are the head of the household
indiv[indiv['hhid'].isin(missing_hoh)].groupby('hhid')['age'].max()

idx=indiv[indiv['hhid'].isin(missing_hoh)].groupby('hhid')['age'].max().index
missing_hoh_df = indiv[indiv['hhid'].isin(idx)].set_index('hhid')
hhdf[hhdf['hhid'].isin(idx)]
hhdf['hoh_mainactivity'] = hhdf['hoh_mainactivity'].fillna(hhdf['hhid'].map(missing_hoh_df['employ'].to_dict()))
hhdf['hoh_education'] = hhdf['hoh_education'].fillna(hhdf['hhid'].map(missing_hoh_df['educ'].to_dict()))


hhdf['inc_nonwage'] = hhdf[['inc_transfer', 'inc_commerce', 'inc_rent','inc_remit', 'inc_gift', 'inc_subsist']].sum(axis=1)

#df['pcwgt'] = df.eval('hhwgt*hhsize')  # population represented
hhdf['pcwgt'] = hhdf['totweight']*hhdf['hhsize']

hhdf = hhdf.rename(columns={'totweight':'hhwgt'})

## final households dataframe with all data complete. 
hhdf.info()

hhdf['pov_line']= [2179 for i in range(len(hhdf))]
hhdf.to_csv('./in/hhdf.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   hhid                         6000 non-null   object 
 1   hhwgt                        6000 non-null   float64
 2   urban                        6000 non-null   int64  
 3   inc_employ                   6000 non-null   float64
 4   inc_commerce                 6000 non-null   float64
 5   inc_subsist                  6000 non-null   float64
 6   inc_remit                    6000 non-null   float64
 7   inc_transfer                 6000 non-null   float64
 8   inc_gift                     6000 non-null   float64
 9   inc_rent                     6000 non-null   float64
 10  inc_total                    6000 non-null   float64
 11  food_cons                    6000 non-null   float64
 12  total_cons                   6000 non-null   float64
 13  poor_nat          

/var/folders/0n/my5lcdhs2xs75tc9zh3p4ms00000gn/T/ipykernel_34263/2300085392.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  indiv[indiv['hhid'].isin(missing_hoh)][indiv['indid']==1]


In [5]:
# useful variables and dictionaries

## educ: ['At least some secondary completion','Primary completion and below','Post-secondary/tertiary and above']
## main_activity: ['Self employment/subsistence', 'Wage employment', 'Not working']
## income main source: ['inc_employ', 'inc_transfer', 'inc_commerce', 'inc_rent','inc_remit', 'inc_gift', 'inc_subsist']

indiv['employ'].unique()

hhdf.head(2)

#income streams:
income_dict ={'inc_employ':'wages',
'inc_commerce':'selfemployed',
'inc_subsist':'subsistence',
'inc_remit':'remittances',
'inc_transfer': 'transfer',
'inc_gift':'windfall',
'inc_rent': 'rental',
'inc_total': 'total_income'
}

consumption_dict = {
    'food_cons':'food',
    'total_cons':'total_consumption'
}


In [6]:
##                         further preparing data to get income profiling figures

## MAIN ACTIVITY
# hoh_mainactivity_2 = [(0,(hh_df_in['hoh_mainactivity']=='private_employee'),'private employees'),
#                                  (1,(hh_df_in['hoh_mainactivity']=='unemployed'),'unemployed')]

## we currently have:  main_activity= ['Self employment/subsistence', 'Wage employment', 'Not working']

# editing to: 
# hoh_mainactivity_2 = [(0,(hh_df_in['hoh_mainactivity']=='Self employment/subsistence'),'self-employed'),
#                                   (1,(hh_df_in['hoh_mainactivity']=='Not working'),'unemployed')]


## EDUCATION LEVEL OF HEAD OF HOUSEHOLD
#'hoh_education':[(0,(hh_df_in['hoh_education']=='less_than_high_school'),'no high school diploma'),
#                             (1,(hh_df_in['hoh_education']!='less_than_high_school'),'at least GCE/HS diploma')]

# we have: educ= ['At least some secondary completion','Primary completion and below','Post-secondary/tertiary and above']

#editing to:
#'hoh_education':[(0,(hh_df_in['hoh_education']=='Primary completion and below'),'no high school diploma'),
#                           (1,(hh_df_in['hoh_education']!='At least some secondary completion'),'at least GCE/HS diploma')]



## INCOME MAIN SOURCE 
#        'hhinc_mainsource_1':[(0,(hh_df_in['hhinc_mainsource']=='wages'),'wages'),
 #                               (1,(hh_df_in['hhinc_mainsource']=='other'),'other')],
  #          'hhinc_mainsource_2':[(0,(hh_df_in['hhinc_mainsource']=='wages'),'wages'),
   #                             (1,(hh_df_in['hhinc_mainsource']=='netirr'),'irregular earnings')]}

# we have: 
## income main source: ['inc_employ', 'inc_transfer', 'inc_commerce', 'inc_rent','inc_remit', 'inc_gift', 'inc_subsist']

#editing to:
#        'hhinc_mainsource_1':[(0,(hh_df_in['hhinc_mainsource']=='inc_employ'),'wages'),
 #                               (1,(hh_df_in['hhinc_mainsource']=='other'),'other')],
  #          'hhinc_mainsource_2':[(0,(hh_df_in['hhinc_mainsource']=='inc_employ'),'wages'),
   #                             (1,(hh_df_in['hhinc_mainsource']=='inc_gift'),'irregular earnings')]}



# Income profiling

In [6]:
from income_profiling_libraries import *

In [7]:
plot_income_profile_side_by_side(hhdf,'hoh_mainactivity_2',xax='pcexp',district=None)
plot_income_profile_side_by_side(hhdf,'hoh_education',xax='pcexp',district=None)
plot_income_profile_side_by_side(hhdf,'hhinc_mainsource_1',xax='pcexp',district=None)
plot_income_profile_side_by_side(hhdf,'hhinc_mainsource_2',xax='pcexp',district=None)

/Users/aarthisundaram/Desktop/Projects/World Bank/Project 3 Fiji Social Protection/fiji_code/income_profiling_libraries.py:95: RuntimeWarning: invalid value encountered in double_scalars
  is_hgt[_i].append(float(hh_df.loc[hh_df.eval(bin_slice)].eval('1E2*pcwgt*{}'.format(_i)).sum()
/Users/aarthisundaram/Desktop/Projects/World Bank/Project 3 Fiji Social Protection/fiji_code/income_profiling_libraries.py:99: RuntimeWarning: invalid value encountered in double_scalars
  ratio.append(float(hh_df.loc[hh_df.eval(bin_slice)].eval('pcwgt*{}'.format(_num)).sum()
/Users/aarthisundaram/Desktop/Projects/World Bank/Project 3 Fiji Social Protection/fiji_code/income_profiling_libraries.py:95: RuntimeWarning: invalid value encountered in double_scalars
  is_hgt[_i].append(float(hh_df.loc[hh_df.eval(bin_slice)].eval('1E2*pcwgt*{}'.format(_i)).sum()
/Users/aarthisundaram/Desktop/Projects/World Bank/Project 3 Fiji Social Protection/fiji_code/income_profiling_libraries.py:99: RuntimeWarning: invalid valu

# Calculating Elasticities

In [9]:
from elasticity_libraries import *

plot_elasticity_regressions(nsims=10000,itoc='REL')


PAY ATTENTION HERE:
-- dropping 0.0% of population b/c (inc<=0 | exp<=0)
- 0 of 10k nsims
- 1 of 10k nsims
- 2 of 10k nsims
- 3 of 10k nsims
- 4 of 10k nsims
- 5 of 10k nsims
- 6 of 10k nsims
- 7 of 10k nsims
- 8 of 10k nsims
- 9 of 10k nsims
